# Bike Sharing Anaysis

##### Problem Statement:

BoomBikes, a US bike-sharing provider, has experienced a decline in revenue due to the COVID-19 pandemic and the accompanying lockdowns. To adapt and prepare for post-quarantine operations, the company is focusing on understanding the demand for shared bikes in the American market. They have enlisted a consulting firm to analyze various factors that influence bike-sharing demand. This includes analyzing significant variables that predict demand and understanding how these variables correlate with bike usage. The aim is to gather insights from a large dataset, which includes daily bike demand influenced by meteorological factors and user preferences. The ultimate business goal is to create a model that predicts shared bike demand. This model will inform management's business strategies, helping them meet customer expectations and adapt to new market dynamics.

##### Strategy:

We can break the problem down into 3 distinct steps in order to solve it.
- Data understanding, cleaning and exploration
- Data modelling and model tuning
- Prediction and inference

##### End objectives:

BoomBikes wants to know the following:
- Which factors are significant predictors of bike sharing demand?
- How well these factors actually predict the demand?

### Part I: Data reading, exploration and EDA

In [11]:
import numpy as np, pandas as pd
data = pd.read_csv('day.csv')
data.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


Let us check whether this dataset contains any missing values.

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    int64  
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.4+ KB


There are no missing values in the data. 

We do not need the column `instant`, since it indicates the record index and nothing useful. So we can drop it.

In [13]:
data.drop('instant', axis = 1, inplace = True)
data.head()

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


Next, we identify the target variable as `cnt`, indicating total count of bike rentals, since this effectively quantifies the demand. This is a sum of the columns `casual` and `registered`. We have to drop the latter two columns because we need to predict the demand and `casual` and `registered` are causally related to `cnt`.

In [14]:
data.drop(['casual', 'registered'], axis = 1, inplace = True)
data.head()

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,985
1,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,801
2,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,1349
3,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,1562
4,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,1600


Next we encode the column `yr` as 2018 and 2019 in order to make it more interpretable.

In [15]:
data[['yr']] = data[['yr']].apply(lambda x: x.map({0: 2018, 1: 2019}))
data.head()

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,01-01-2018,1,2018,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,985
1,02-01-2018,1,2018,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,801
2,03-01-2018,1,2018,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,1349
3,04-01-2018,1,2018,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,1562
4,05-01-2018,1,2018,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,1600


Since we already have year, month, weekday/workday and season information, we can extract day of month from `dteday` column and drop it.

In [23]:
data['dteday'] = pd.to_datetime(data['dteday'], format = "%d-%m-%Y")
data['dayofmonth'] = data['dteday'].dt.day
data.drop('dteday', axis = 1, inplace = True)
data.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt,dayofmonth
0,1,2018,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,985,1
1,1,2018,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,801,2
2,1,2018,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,1349,3
3,1,2018,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,1562,4
4,1,2018,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,1600,5
